In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set(style='whitegrid')
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,4)

In [2]:
fs = 200.0         # sampling frequency (Hz)
T = 5.0            # duration (s)
t = np.arange(0, T, 1/fs)

#
f1, f2 = 5.0, 30.0
sig = 0.8*np.sin(2*np.pi*f1*t) + 0.4*np.sin(2*np.pi*f2*t)
sig += 0.02*t      # 
# 
sig += np.where((t>2.0) & (t<2.02), 2.0, 0.0)

In [3]:
np.random.seed(0)
noise = 0.5*np.random.randn(len(t))     # Gaussian noise
# 
outliers = np.zeros_like(t)
outliers[[100, 400, 700]] = [5.0, -4.0, 6.0]
sig_noisy = sig + noise + outliers

plt.plot(t, sig_noisy, label='Noisy signal')
plt.plot(t, sig, label='Clean (ground truth)', alpha=0.6)
plt.legend(); plt.xlabel('Time [s]'); plt.ylabel('Amplitude')

Text(0, 0.5, 'Amplitude')

In [4]:
f, Pxx = signal.welch(sig_noisy, fs=fs, nperseg=256)
plt.semilogy(f, Pxx)
plt.xlabel('Frequency [Hz]'); plt.ylabel('PSD')


Text(88.875, 0.5, 'PSD')

In [5]:
def moving_average(x, N):
    return np.convolve(x, np.ones(N)/N, mode='same')

ma = moving_average(sig_noisy, N=7)
plt.plot(t, sig_noisy, alpha=0.4, label='Noisy')
plt.plot(t, ma, label='Moving avg (N=7)')
plt.plot(t, sig, label='Clean', alpha=0.7)
plt.legend()

In [6]:
# 
cutoff = 15.0
numtaps = 101
h = signal.firwin(numtaps, cutoff/(fs/2))
sig_fir = signal.lfilter(h, [1.0], sig_noisy)

plt.plot(t, sig_noisy, alpha=0.4, label='Noisy')
plt.plot(t, sig_fir, label='FIR filtered')
plt.plot(t, sig, label='Clean', alpha=0.7)
plt.legend()

In [7]:
b, a = signal.butter(4, cutoff/(fs/2), btype='low')
sig_iir = signal.filtfilt(b, a, sig_noisy)   # zero-phase

plt.plot(t, sig_noisy, alpha=0.4, label='Noisy')
plt.plot(t, sig_iir, label='IIR (butter, filtfilt)')
plt.plot(t, sig, label='Clean', alpha=0.7)
plt.legend()

In [8]:
def mse(x,y): return np.mean((x-y)**2)
def snr_db(clean, noisy):
    P_signal = np.mean(clean**2)
    P_noise = np.mean((clean-noisy)**2)
    return 10*np.log10(P_signal / P_noise)

print("MSE noisy:", mse(sig, sig_noisy))
print("MSE FIR:", mse(sig, sig_fir))
print("MSE IIR:", mse(sig, sig_iir))
print("SNR noisy:", snr_db(sig, sig_noisy))
print("SNR FIR:", snr_db(sig, sig_fir))
print("SNR IIR:", snr_db(sig, sig_iir))

MSE noisy: 0.3242113845097848
MSE FIR: 0.7760780588719283
MSE IIR: 0.13180909680718866
SNR noisy: 1.1342133216838954
SNR FIR: -2.6565581235291242
SNR IIR: 5.04304208833843


In [9]:
f, t_spec, Sxx = signal.spectrogram(sig_noisy, fs=fs, nperseg=128, noverlap=64)
plt.pcolormesh(t_spec, f, 10*np.log10(Sxx), shading='gouraud')
plt.ylabel('Frequency [Hz]'); plt.xlabel('Time [s]')
plt.colorbar(label='PSD [dB]')

In [10]:
# sliding window features
window = int(0.5*fs)  # 0.5 s
features = []
for i in range(0, len(sig_noisy)-window, window):
    w = sig_noisy[i:i+window]
    features.append({
        'mean': np.mean(w),
        'rms': np.sqrt(np.mean(w**2)),
        'std': np.std(w),
        'peak_to_peak': np.ptp(w)
    })
df_feat = pd.DataFrame(features)
df_feat.head()

,mean,rms,std,peak_to_peak
0,0.136504,0.855588,0.844628,3.473394
1,0.004307,0.994383,0.994374,7.699790
2,0.096984,0.840857,0.835246,4.266534
3,-0.165098,0.816686,0.799824,3.795027
4,0.160318,1.044655,1.032280,7.310495
